In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [75]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [76]:
data_directory = '/content/drive/MyDrive/Grape'

In [77]:
all_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [78]:
train_generator = all_data_gen.flow_from_directory(
    directory=data_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    color_mode='rgb',
    subset='training'
)

Found 3275 images belonging to 4 classes.


In [79]:
validation_generator = all_data_gen.flow_from_directory(
    directory=data_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation'
)

Found 817 images belonging to 4 classes.


In [80]:
test_split_gen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
test_generator = test_split_gen.flow_from_directory(
    directory=data_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation',
    shuffle=False
)

Found 1022 images belonging to 4 classes.


In [81]:
steps_per_epoch = len(data_directory) // 16
steps_per_epoch

1

In [82]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),  # Change input size to (256, 256, 3)
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)                   │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_29 (MaxPooling2D)      │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_30 (Conv2D)                   │ (None, 127, 127, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_30 (MaxPooling2D)      │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 63, 63, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_31 (MaxPooling2D)      │ (None, 31, 31, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_32 (Conv2D)                   │ (None, 31, 31, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_32 (MaxPooling2D)      │ (None, 15, 15, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_33 (Conv2D)                   │ (None, 15, 15, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_33 (MaxPooling2D)      │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_34 (Conv2D)                   │ (None, 7, 7, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_34 (MaxPooling2D)      │ (None, 3, 3, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 185,700 (725.39 KB)

 Trainable params: 185,700 (725.39 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics  = ['accuracy']
    )



In [86]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Correct calculation for steps per epoch
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # Correct calculation for validation steps
    epochs=6
)


Epoch 1/6
204/204 ━━━━━━━━━━━━━━━━━━━━ 535s 3s/step - accuracy: 0.8191 - loss: 0.4567 - val_accuracy: 0.8775 - val_loss: 0.3174
Epoch 2/6
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 258us/step - accuracy: 0.7500 - loss: 0.3427 - val_accuracy: 1.0000 - val_loss: 0.0985
Epoch 3/6
 49/204 ━━━━━━━━━━━━━━━━━━━━ 8:41 3s/step - accuracy: 0.8738 - loss: 0.2898

KeyboardInterrupt: 

In [44]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

64/64 ━━━━━━━━━━━━━━━━━━━━ 46s 716ms/step - accuracy: 0.8993 - loss: 0.2984
Test Accuracy: 88.26%


In [45]:
model.save('LDD_grapes_model.h5')

In [ ]:
import os
model_path = 'LDD_grapes_model.h5'
if os.path.exists(model_path):
    print("Model is saved.")
else:
    print("Model is not saved.")

In [ ]:
model.save('model.h5')


In [ ]:
from tensorflow.keras.models import load_model

model_path = '/content/model.h5'
model = load_model(model_path)

# Print the input shape of the model
print(model.input_shape)


In [47]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input  # Add this line

# Load the saved model
model = load_model('/content/LDD_grapes_model.h5')

# Define class labels
class_labels = [
    'Black Rot',
    'Esca (Black Measles)',
    'Leaf Blight (Isariopsis Leaf Spot)',
    'Healthy'
]

# Load an image for prediction
img_path = '/content/grape.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)  # This preprocesses the input for InceptionV3

# Predict the class
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]
predicted_class_label = class_labels[predicted_class_index]

# Display the result
print(f"Predicted Class: {predicted_class_label}")

# Recommendation based on the prediction
recommendations = {
    'Black Rot': "Remove and destroy infected leaves or fruit to reduce fungal spread. Apply fungicides such as mancozeb or myclobutanil during the growing season.",
    'Esca (Black Measles)': "Prune infected wood during the dormant season. Avoid injuring the vines to prevent fungal entry. If infection is severe, consider removing the vine.",
    'Leaf Blight (Isariopsis Leaf Spot)': "Improve air circulation around the grapevine by pruning overcrowded leaves. Apply a copper-based fungicide to control the spread.",
    'Healthy': "Maintain good cultural practices such as regular watering, proper fertilization, and pest monitoring to sustain the vine's health."
}

print(f"Recommendation: {recommendations[predicted_class_label]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Predicted Class: Black Rot
Recommendation: Remove and destroy infected leaves or fruit to reduce fungal spread. Apply fungicides such as mancozeb or myclobutanil during the growing season.
